### TODO
<ul>
<li> Currently, we are dealing with only deterministic policies. Have to extend the implementation to stochastic policies. </li>
</ul>

In [48]:
import numpy as np

np.set_printoptions(suppress=True)

class GridWorld:
    def __init__(self,GRID_SIZE=50):
        self.grid_size = GRID_SIZE
        self.num_states = self.grid_size**2
        self.rewards = np.random.choice([0,-1],p=[0.67,0.33],size=(self.grid_size,self.grid_size))
        self.rewards[0,self.grid_size-1] = 1
        self.goal_state = self.grid_size-1
        self.actions = np.array(["up","down","left","right"])
        
    def get_feature(self,state):
        _ = [0]*self.num_states
        _[state] = 1
        return _
        
    
    def get_rewards(self):
        return self.rewards.flatten()
    
    def result_of_action(self,state,action):
        state_coords = (state/self.grid_size,state%self.grid_size)
        next_states = [(max(0,state_coords[0]-1),state_coords[1]),(min(self.grid_size-1,state_coords[0]+1),state_coords[1]),\
                      (state_coords[0],max(0,state_coords[1]-1)),(state_coords[0],min(self.grid_size-1,state_coords[1]+1))]
        transition_probs = 0.1*np.ones((len(self.actions)))
        transition_probs[np.where(self.actions == action)[0][0]] = 0.7
        next_state = next_states[np.random.choice(range(len(next_states)),p=transition_probs)]
        return next_state[0]*self.grid_size+next_state[1]
    
    def generate_trajectory(self,policy=None,num_trajectories=10):
        if policy is None:
            policy = np.random.choice(self.actions,size=(self.num_states))
        trajectories = []
        for i in range(num_trajectories):
            trajectory = []
            current_state = np.random.randint(self.num_states)
            while current_state != self.goal_state and len(trajectory) < self.grid_size*3:
                trajectory.append(self.get_feature(current_state))
                current_state = self.result_of_action(current_state,policy[current_state])
            if current_state == self.goal_state:
                trajectory.append(self.get_feature(self.goal_state))
            trajectories.append(np.array(trajectory))
        return np.array(trajectories)
        
    
    def get_transition_probabilities(self,state,action):
        '''While calculating the transition probabilities, we make the assumption that if you were in a cell along
        the border, and you tried to make a transition outside the border with probability p, you end up not
        moving with the same probability p.'''
        transition_probs = np.zeros((self.grid_size,self.grid_size))
        state_coords = (state/self.grid_size,state%self.grid_size)
        if action == "up":
            transition_probs[max(0,state_coords[0]-1),state_coords[1]] += 0.7 # up
            transition_probs[state_coords[0],max(0,state_coords[1]-1)] += 0.1 # left
            transition_probs[min(self.grid_size-1,state_coords[0]+1),state_coords[1]] += 0.1 # down
            transition_probs[state_coords[0],min(self.grid_size-1,state_coords[1]+1)] += 0.1 #right
        elif action == "down":
            transition_probs[max(0,state_coords[0]-1),state_coords[1]] += 0.1
            transition_probs[state_coords[0],max(0,state_coords[1]-1)] += 0.1
            transition_probs[min(self.grid_size-1,state_coords[0]+1),state_coords[1]] += 0.7
            transition_probs[state_coords[0],min(self.grid_size-1,state_coords[1]+1)] += 0.1
        elif action == "left":
            transition_probs[max(0,state_coords[0]-1),state_coords[1]] += 0.1
            transition_probs[state_coords[0],max(0,state_coords[1]-1)] += 0.7
            transition_probs[min(self.grid_size-1,state_coords[0]+1),state_coords[1]] += 0.1
            transition_probs[state_coords[0],min(self.grid_size-1,state_coords[1]+1)] += 0.1
        elif action == "right":
            transition_probs[max(0,state_coords[0]-1),state_coords[1]] += 0.1
            transition_probs[state_coords[0],max(0,state_coords[1]-1)] += 0.1
            transition_probs[min(self.grid_size-1,state_coords[0]+1),state_coords[1]] += 0.1
            transition_probs[state_coords[0],min(self.grid_size-1,state_coords[1]+1)] += 0.7
        return transition_probs.flatten()
    
    def take_greedy_action(self,values):
        values = values.reshape(self.grid_size,self.grid_size)
        policy = np.repeat("random",self.num_states)
        for i in range(self.num_states):
            state_coords = (i/self.grid_size,i%self.grid_size)
            policy[i] = self.actions[np.argmax([values[max(0,state_coords[0]-1),state_coords[1]],
                                                values[min(self.grid_size-1,state_coords[0]+1),state_coords[1]],
                                                values[state_coords[0],max(0,state_coords[1]-1)],
                                                values[state_coords[0],min(self.grid_size-1,state_coords[1]+1)]])]
        return policy
            


In [49]:
class PolicyIteration:
    def __init__(self,env):
        self.env = env
        self.values = np.zeros((self.env.num_states,))
        self.policy = np.random.choice(self.env.actions,size=(self.env.num_states))
    
    def policy_evaluation(self,num_iters=10,gamma=0.99):
        for i in range(num_iters):
            transition_probs = np.zeros((self.env.num_states,self.env.num_states))
            for j in range(self.env.num_states):
                transition_probs[j] = self.env.get_transition_probabilities(j,self.policy[j])
            self.values = self.env.get_rewards() + gamma*np.dot(transition_probs,self.values)
    
    def policy_iteration(self,num_iters=10):
        for i in range(num_iters):
            self.policy_evaluation()
            self.policy = self.env.take_greedy_action(self.values)
        return self.policy
        

gw = GridWorld(10)
print gw.rewards
pi = PolicyIteration(gw)
optimal_policy = pi.policy_iteration(100)


[[-1  0  0 -1  0 -1 -1 -1 -1  1]
 [ 0  0  0  0 -1  0 -1  0  0 -1]
 [ 0 -1 -1  0  0 -1  0 -1  0  0]
 [ 0 -1 -1  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0 -1 -1  0 -1]
 [ 0  0  0 -1  0  0  0 -1  0 -1]
 [ 0  0 -1  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0 -1  0 -1  0 -1]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 -1 -1  0  0  0]]


In [71]:
np.set_printoptions(threshold=np.nan)

def feature_averages(trajectory,gamma=0.99):
    horizon = len(trajectory)
    return np.sum(np.multiply(trajectory,np.array([gamma**j for j in range(horizon)]).reshape(horizon,1)),axis=0)

class RelEntIRL:
    def __init__(self,expert_demos,nonoptimal_demos):
        self.expert_demos = expert_demos
        self.nonoptimal_demos = nonoptimal_demos
        self.num_features = len(self.expert_demos[0][0])
        self.weights = np.zeros((self.num_features,))
    
    def calculate_expert_feature(self):
        self.expert_feature = np.zeros_like(self.weights)
        for i in range(len(self.expert_demos)):
            self.expert_feature += feature_averages(self.expert_demos[i])
        self.expert_feature /= len(self.expert_demos)
        return self.expert_feature
    
    def train(self,num_iters=10000,step_size=1e-4):
        self.calculate_expert_feature()
        self.policy_features = np.zeros((len(self.nonoptimal_demos),self.num_features))
        for i in range(len(self.nonoptimal_demos)):
            self.policy_features[i] = feature_averages(self.nonoptimal_demos[i])
            
        importance_sampling = np.zeros((len(self.nonoptimal_demos),))
        for i in range(num_iters):
            update = np.zeros_like(self.weights)
            for j in range(len(self.nonoptimal_demos)):
                importance_sampling[j] = np.exp(np.dot(self.policy_features[j],self.weights))
            importance_sampling /= np.sum(importance_sampling,axis=0)
            weighted_sum = np.sum(np.multiply(np.array([importance_sampling,]*self.policy_features.shape[1]).T,\
                                              self.policy_features),axis=0)
            self.weights += step_size*(self.expert_feature - weighted_sum)
                
    
expert_trajectories = gw.generate_trajectory(optimal_policy)
nonoptimal_trajectories = gw.generate_trajectory()
relent = RelEntIRL(expert_trajectories,nonoptimal_trajectories)
relent.train()
print relent.weights.reshape(10,10)

[[-0.12622733 -0.10404033  0.          0.         -0.22384805 -0.46498198
  -0.21502754 -0.74728053  0.09801     0.87161908]
 [-0.86868083 -0.16352218 -0.16188696 -0.16026809 -0.15866541 -0.15262418
   0.09227447 -0.30311082  0.39517875  1.29766903]
 [-0.81787899 -0.23538422  0.          0.          0.          0.          0.1
  -0.07586245  0.33762384  0.25980982]
 [ 0.         -0.4831602   0.          0.          0.45344428  0.82225253
   0.27893693 -0.45360797 -0.39070429  0.        ]
 [ 0.         -0.67250064  0.09813165  0.28936172  0.72937569  0.56270725
  -0.18012098 -0.84448055 -0.87283156  0.        ]
 [-0.28168265 -0.07602425 -0.42437219 -0.43680115  0.64427006  0.194099
  -0.09727599 -0.61002607 -0.28081265 -0.21165468]
 [-0.55766636 -0.44222197 -0.68270406 -0.56360288  0.13821161  0.          0.
  -0.44946401 -0.19821716 -0.16545535]
 [-0.10217138 -0.3769091  -0.57448197  0.04985926 -0.04963934  0.          0.
  -0.22796876 -0.69380162 -0.78261119]
 [-0.87916683 -0.41805071